<a href="https://colab.research.google.com/github/msudhanshu10/Data-Science/blob/master/Advanced%20House%20Price%20Prediction/Advanced_House_Price_Prediction_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import files
uploaded= files.upload()

Saving test.csv to test.csv


In [4]:
dataset= pd.read_csv('test.csv')

In [5]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## 1. Handling Missing Values

1.1 Handle missing values for categorical features.

In [6]:
isnull_cat_features= [feature for feature in dataset.columns if dataset[feature].isna().sum()>1 and dataset[feature].dtypes == 'O']

for feature in isnull_cat_features:
  print('Missing values in',feature, '->', dataset[feature].isna().mean()*100)

Missing values in MSZoning -> 0.2741603838245374
Missing values in Alley -> 92.66620973269363
Missing values in Utilities -> 0.1370801919122687
Missing values in MasVnrType -> 1.0966415352981496
Missing values in BsmtQual -> 3.015764222069911
Missing values in BsmtCond -> 3.0843043180260454
Missing values in BsmtExposure -> 3.015764222069911
Missing values in BsmtFinType1 -> 2.878684030157642
Missing values in BsmtFinType2 -> 2.878684030157642
Missing values in Functional -> 0.1370801919122687
Missing values in FireplaceQu -> 50.03427004797807
Missing values in GarageType -> 5.20904729266621
Missing values in GarageFinish -> 5.346127484578479
Missing values in GarageQual -> 5.346127484578479
Missing values in GarageCond -> 5.346127484578479
Missing values in PoolQC -> 99.7943797121316
Missing values in Fence -> 80.12337217272105
Missing values in MiscFeature -> 96.50445510623715


In [7]:
def replace_nan_in_catfeatures(dataset, features_nan):
  data= dataset.copy()
  data[features_nan]= data[features_nan].fillna('Missing')
  return data

In [9]:
dataset= replace_nan_in_catfeatures(dataset, isnull_cat_features)

dataset[isnull_cat_features].isna().mean()

MSZoning        0.0
Alley           0.0
Utilities       0.0
MasVnrType      0.0
BsmtQual        0.0
BsmtCond        0.0
BsmtExposure    0.0
BsmtFinType1    0.0
BsmtFinType2    0.0
Functional      0.0
FireplaceQu     0.0
GarageType      0.0
GarageFinish    0.0
GarageQual      0.0
GarageCond      0.0
PoolQC          0.0
Fence           0.0
MiscFeature     0.0
dtype: float64

1.2 Handling Numerical features having nan values

In [10]:
isnull_num_features= [feature for feature in dataset.columns if dataset[feature].isna().sum()>1 and dataset[feature].dtypes != 'O']

for feature in isnull_num_features:
  print('Missing values in',feature, '->', dataset[feature].isna().mean()*100)

Missing values in LotFrontage -> 15.558601782042494
Missing values in MasVnrArea -> 1.0281014393420151
Missing values in BsmtFullBath -> 0.1370801919122687
Missing values in BsmtHalfBath -> 0.1370801919122687
Missing values in GarageYrBlt -> 5.346127484578479


In [11]:
## Replacing with the median

def replace_nan_in_numfeatures(dataset, features_nan):
  data= dataset.copy()
  for feature in features_nan:
    median_value= data[feature].median()
    data[feature+'isnan']= np.where(data[feature].isna(), 1, 0)
    data[feature].fillna(median_value, inplace= True)

  return data

In [12]:
dataset= replace_nan_in_numfeatures(dataset, isnull_num_features)

dataset[isnull_num_features].isna().sum()


LotFrontage     0
MasVnrArea      0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
dtype: int64

## 2. Handling Temporal Data

In [13]:
numeric_features= [feature for feature in dataset.columns if dataset[feature].dtypes != 'O']

In [14]:
temporal_features= [feature for feature in numeric_features if 'Yr' in feature or 'Year' in feature]

for feature in temporal_features:
  if feature != 'YrSold':
    dataset[feature]= dataset['YrSold']- dataset[feature]


dataset[temporal_features].head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,GarageYrBltisnan
0,49,49,49.0,2010,2010
1,52,52,52.0,2010,2010
2,13,12,13.0,2010,2010
3,12,12,12.0,2010,2010
4,18,18,18.0,2010,2010


In [15]:
dataset.drop('GarageYrBltisnan', axis=1, inplace=True)

In [16]:
temporal_features=['YearBuilt',	'YearRemodAdd',	'GarageYrBlt',	'YrSold']
dataset[temporal_features].head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,49,49,49.0,2010
1,52,52,52.0,2010
2,13,12,13.0,2010
3,12,12,12.0,2010
4,18,18,18.0,2010


## 3. Numeric Variables
Since, the distribution of the continuous numeric features was skewed, we will use log normal transformation.

In [18]:
## Taking only those values which do not have zeros.
num_features= ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']
for feature in num_features:
   dataset[feature]= np.log(dataset[feature])

## 4. Handling Rare categorical features.
A category/class is called as rare if it is present very less in number, say 1%. These categories will not add value to the dependent feature, so we will club them to a new category.

In [19]:
cat_features= [feature for feature in dataset.columns if dataset[feature].dtypes == 'O']
print('Count of Categorical Features  ',len(cat_features))
dataset[cat_features].head(3)

Count of Categorical Features   43


,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RH,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,CBlock,TA,TA,No,Rec,LwQ,GasA,TA,Y,SBrkr,TA,Typ,Missing,Attchd,Unf,TA,TA,Y,Missing,MnPrv,Missing,WD,Normal
1,RL,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,Gd,Typ,Missing,Attchd,Unf,TA,TA,Y,Missing,Missing,Gar2,WD,Normal
2,RL,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,Attchd,Fin,TA,TA,Y,Missing,MnPrv,Missing,WD,Normal


In [20]:
len(dataset)

1459

In [22]:
temp= dataset.groupby('Alley')['Id'].count()/len(dataset)
temp

Alley
Grvl       0.047978
Missing    0.926662
Pave       0.025360
Name: Id, dtype: float64

In [23]:
temp[temp<0.7].index

Index(['Grvl', 'Pave'], dtype='object', name='Alley')

In [24]:
for feature in cat_features:
  temp= dataset.groupby(feature)['Id'].count()/len(dataset)

  temp_df= temp[temp >0.01].index

  dataset[feature]= np.where(dataset[feature].isin(temp_df), dataset[feature], 'Rare_val')

In [25]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontageisnan,MasVnrAreaisnan,BsmtFullBathisnan,BsmtHalfBathisnan
0,1461,20,Rare_val,1.477511,2.236515,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,49,49,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,...,0,0,1.916620,0.0,0.0,1,0,2,1,TA,5,Typ,0,Missing,Attchd,49.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,Missing,MnPrv,Missing,0,6,2010,WD,Normal,0,0,0,0
1,1462,20,RL,1.480342,2.258184,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,52,52,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,...,0,0,1.972995,0.0,0.0,1,1,3,1,Gd,6,Typ,0,Missing,Attchd,52.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,Missing,Missing,Rare_val,12500,6,2010,WD,Normal,0,0,0,0
2,1463,60,RL,1.459560,2.254927,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,13,12,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,...,701,0,2.000902,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,13.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,Missing,MnPrv,Missing,0,3,2010,WD,Normal,0,0,0,0
3,1464,60,RL,1.471717,2.220088,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,12,12,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,...,678,0,1.998808,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,12.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,Missing,Missing,Missing,0,6,2010,WD,Normal,0,0,0,0
4,1465,120,RL,1.324738,2.142204,Pave,Missing,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,18,18,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,...,0,0,1.967758,0.0,0.0,2,0,2,1,Gd,5,Typ,0,Missing,Attchd,18.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,Missing,Missing,Missing,0,1,2010,WD,Normal,0,0,0,0


## 5. Label Encoding

In [26]:
cat_features

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [27]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

for feature in cat_features:
  dataset[feature]= le.fit_transform(dataset[feature])

dataset.head(50)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontageisnan,MasVnrAreaisnan,BsmtFullBathisnan,BsmtHalfBathisnan
0,1461,20,4,1.477511,2.236515,0,1,3,3,0,3,0,9,1,0,0,1,5,6,49,49,0,0,8,9,2,0.0,3,3,1,4,4,4,5,468.0,3,144.0,270.0,882.0,0,...,0,0,1.916620,0.0,0.0,1,0,2,1,4,5,4,0,3,1,49.0,3,1.0,730.0,3,3,2,140,0,0,0,120,0,0,3,0,0,6,2010,4,2,0,0,0,0
1,1462,20,2,1.480342,2.258184,0,1,0,3,0,0,0,9,2,0,0,1,6,6,52,52,1,0,9,10,0,108.0,3,3,1,4,4,4,0,923.0,6,0.0,406.0,1329.0,0,...,0,0,1.972995,0.0,0.0,1,1,3,1,2,6,4,0,3,1,52.0,3,1.0,312.0,3,3,2,393,36,0,0,0,0,0,2,1,12500,6,2010,4,2,0,0,0,0
2,1463,60,2,1.459560,2.254927,0,1,0,3,0,3,0,5,2,0,0,2,5,5,13,12,0,0,8,9,2,0.0,3,3,2,2,4,4,2,791.0,6,0.0,137.0,928.0,0,...,701,0,2.000902,0.0,0.0,2,1,3,1,4,6,4,1,5,1,13.0,0,2.0,482.0,3,3,2,212,34,0,0,0,0,0,3,0,0,3,2010,4,2,0,0,0,0
3,1464,60,2,1.471717,2.220088,0,1,0,3,0,3,0,5,2,0,0,2,6,6,12,12,0,0,8,9,0,20.0,3,3,2,4,4,4,2,602.0,6,0.0,324.0,926.0,0,...,678,0,1.998808,0.0,0.0,2,1,3,1,2,7,4,1,2,1,12.0,0,2.0,470.0,3,3,2,360,36,0,0,0,0,0,2,0,0,6,2010,4,2,0,0,0,0
4,1465,120,2,1.324738,2.142204,0,1,0,1,0,3,0,19,2,0,4,1,8,5,18,18,0,0,3,4,2,0.0,2,3,2,2,4,4,0,263.0,6,0.0,1017.0,1280.0,0,...,0,0,1.967758,0.0,0.0,2,0,2,1,2,5,4,0,3,1,18.0,2,2.0,506.0,3,3,2,0,82,0,0,144,0,0,2,0,0,1,2010,4,2,0,0,0,0
5,1466,60,2,1.462674,2.220327,0,1,0,3,0,0,0,5,2,0,0,2,6,5,17,16,0,0,3,4,2,0.0,3,3,2,2,4,4,6,0.0,6,0.0,763.0,763.0,0,...,892,0,2.003040,0.0,0.0,2,1,3,1,4,7,4,1,5,1,17.0,0,2.0,440.0,3,3,2,157,84,0,0,0,0,0,2,0,0,4,2010,4,2,0,0,0,0
6,1467,20,2,1.436201,2.195522,0,1,0,3,0,3,0,5,2,0,0,1,6,7,18,3,0,0,3,4,2,0.0,3,1,2,2,4,4,0,935.0,6,0.0,233.0,1168.0,0,...,0,0,1.957159,1.0,0.0,2,0,3,1,4,6,4,0,3,1,18.0,0,2.0,420.0,3,3,2,483,21,0,0,0,0,0,0,2,500,3,2010,4,2,1,0,0,0
7,1468,60,2,1.421453,2.201242,0,1,0,3,0,3,0,5,2,0,0,2,6,5,12,12,0,0,8,9,2,0.0,3,3,2,2,4,4,6,0.0,6,0.0,789.0,789.0,0,...,676,0,1.986450,0.0,0.0,2,1,3,1,4,7,4,1,2,1,12.0,0,2.0,393.0,3,3,2,0,75,0,0,0,0,0,2,0,0,5,2010,4,2,0,0,0,0
8,1469,20,2,1.491251,2.222219,0,1,3,3,0,3,0,5,2,0,0,1,7,5,20,20,0,0,3,4,2,0.0,3,3,2,2,4,1,2,637.0,6,0.0,663.0,1300.0,0,...,0,0,1.974244,1.0,0.0,1,1,2,1,2,5,4,1,4,1,20.0,3,2.0,506.0,3,3,2,192,0,0,0,0,0,0,2,0,0,2,2010,4,2,0,0,0,0
9,1470,20,2,1.446565,2.201215,0,1,3,3,0,0,0,9,2,0,0,1,4,5,40,40,0,0,5,6,2,0.0,3,3,1,4,4,4,0,804.0,5,78.0,0.0,882.0,0,...,0,0,1.914300,1.0,0.0,1,0,2,1,4,4,4,0,3,1,40.0,0,2.0,525.0,3,3,2,240,0,0,0,0,0,0,3,0,0,4,2010,4,2,0,0,0,0


## 6. Feature Scaling

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()


In [29]:
features_scale= [feature for feature in dataset.columns if feature not in ['Id', 'SalePrice']]

In [30]:
scaler.fit(dataset[features_scale])

MinMaxScaler(copy=True, feature_range=(0, 1))

MinMaxScaler and StandardScaler both can be used, one should try both and see which gives better r2_score.

In [31]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontageisnan,MasVnrAreaisnan,BsmtFullBathisnan,BsmtHalfBathisnan
0,1461,20,4,1.477511,2.236515,0,1,3,3,0,3,0,9,1,0,0,1,5,6,49,49,0,0,8,9,2,0.0,3,3,1,4,4,4,5,468.0,3,144.0,270.0,882.0,0,...,0,0,1.916620,0.0,0.0,1,0,2,1,4,5,4,0,3,1,49.0,3,1.0,730.0,3,3,2,140,0,0,0,120,0,0,3,0,0,6,2010,4,2,0,0,0,0
1,1462,20,2,1.480342,2.258184,0,1,0,3,0,0,0,9,2,0,0,1,6,6,52,52,1,0,9,10,0,108.0,3,3,1,4,4,4,0,923.0,6,0.0,406.0,1329.0,0,...,0,0,1.972995,0.0,0.0,1,1,3,1,2,6,4,0,3,1,52.0,3,1.0,312.0,3,3,2,393,36,0,0,0,0,0,2,1,12500,6,2010,4,2,0,0,0,0
2,1463,60,2,1.459560,2.254927,0,1,0,3,0,3,0,5,2,0,0,2,5,5,13,12,0,0,8,9,2,0.0,3,3,2,2,4,4,2,791.0,6,0.0,137.0,928.0,0,...,701,0,2.000902,0.0,0.0,2,1,3,1,4,6,4,1,5,1,13.0,0,2.0,482.0,3,3,2,212,34,0,0,0,0,0,3,0,0,3,2010,4,2,0,0,0,0
3,1464,60,2,1.471717,2.220088,0,1,0,3,0,3,0,5,2,0,0,2,6,6,12,12,0,0,8,9,0,20.0,3,3,2,4,4,4,2,602.0,6,0.0,324.0,926.0,0,...,678,0,1.998808,0.0,0.0,2,1,3,1,2,7,4,1,2,1,12.0,0,2.0,470.0,3,3,2,360,36,0,0,0,0,0,2,0,0,6,2010,4,2,0,0,0,0
4,1465,120,2,1.324738,2.142204,0,1,0,1,0,3,0,19,2,0,4,1,8,5,18,18,0,0,3,4,2,0.0,2,3,2,2,4,4,0,263.0,6,0.0,1017.0,1280.0,0,...,0,0,1.967758,0.0,0.0,2,0,2,1,2,5,4,0,3,1,18.0,2,2.0,506.0,3,3,2,0,82,0,0,144,0,0,2,0,0,1,2010,4,2,0,0,0,0


In [32]:
scaler.transform(dataset[features_scale])

array([[0.        , 1.        , 0.65728796, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.66239742, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.5       , 0.62488741, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.5       , 0.92233754, ..., 0.        , 0.        ,
        0.        ],
       [0.38235294, 0.5       , 0.5491235 , ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.5       , 0.62488741, ..., 0.        , 0.        ,
        0.        ]])

In [34]:
dataset_scaled= pd.concat([dataset['Id'].reset_index(drop=True),
                           pd.DataFrame(scaler.transform(dataset[features_scale]), columns=features_scale)],
                           axis=1)

In [35]:
dataset_scaled.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontageisnan,MasVnrAreaisnan,BsmtFullBathisnan,BsmtHalfBathisnan
0,1461,0.000000,1.0,0.657288,0.614997,0.0,0.5,1.0,1.000000,0.0,0.75,0.0,0.45,0.166667,0.0,0.0,0.2,0.444444,0.625,0.384615,0.822581,0.0,0.0,0.8,0.818182,0.5,0.000000,1.000000,1.0,0.25,1.0,1.0,1.0,0.833333,0.116708,0.5,0.094364,0.126168,0.173111,0.0,...,0.000000,0.0,0.351478,0.0,0.0,0.25,0.0,0.333333,0.5,1.0,0.166667,1.0,0.00,0.6,0.166667,0.792994,1.000000,0.2,0.490591,1.0,1.0,1.0,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.0,0.75,0.0,0.000000,0.454545,1.0,1.0,0.5,0.0,0.0,0.0,0.0
1,1462,0.000000,0.5,0.662397,0.668388,0.0,0.5,0.0,1.000000,0.0,0.00,0.0,0.45,0.333333,0.0,0.0,0.2,0.555556,0.625,0.407692,0.870968,0.5,0.0,0.9,0.909091,0.0,0.083721,1.000000,1.0,0.25,1.0,1.0,1.0,0.000000,0.230175,1.0,0.000000,0.189720,0.260844,0.0,...,0.000000,0.0,0.512060,0.0,0.0,0.25,0.5,0.500000,0.5,0.5,0.250000,1.0,0.00,0.6,0.166667,0.802548,1.000000,0.2,0.209677,1.0,1.0,1.0,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.0,0.50,0.5,0.735294,0.454545,1.0,1.0,0.5,0.0,0.0,0.0,0.0
2,1463,0.235294,0.5,0.624887,0.660362,0.0,0.5,0.0,1.000000,0.0,0.75,0.0,0.25,0.333333,0.0,0.0,0.4,0.444444,0.500,0.107692,0.225806,0.0,0.0,0.8,0.818182,0.5,0.000000,1.000000,1.0,0.50,0.5,1.0,1.0,0.333333,0.197257,1.0,0.000000,0.064019,0.182139,0.0,...,0.376477,0.0,0.591552,0.0,0.0,0.50,0.5,0.500000,0.5,1.0,0.250000,1.0,0.25,1.0,0.166667,0.678344,0.000000,0.4,0.323925,1.0,1.0,1.0,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.0,0.75,0.0,0.000000,0.181818,1.0,1.0,0.5,0.0,0.0,0.0,0.0
3,1464,0.235294,0.5,0.646830,0.574520,0.0,0.5,0.0,1.000000,0.0,0.75,0.0,0.25,0.333333,0.0,0.0,0.4,0.555556,0.625,0.100000,0.225806,0.0,0.0,0.8,0.818182,0.0,0.015504,1.000000,1.0,0.50,1.0,1.0,1.0,0.333333,0.150125,1.0,0.000000,0.151402,0.181747,0.0,...,0.364125,0.0,0.585589,0.0,0.0,0.50,0.5,0.500000,0.5,0.5,0.333333,1.0,0.25,0.4,0.166667,0.675159,0.000000,0.4,0.315860,1.0,1.0,1.0,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.0,0.50,0.0,0.000000,0.454545,1.0,1.0,0.5,0.0,0.0,0.0,0.0
4,1465,0.588235,0.5,0.381547,0.382619,0.0,0.5,0.0,0.333333,0.0,0.75,0.0,0.95,0.333333,0.0,1.0,0.2,0.777778,0.500,0.146154,0.322581,0.0,0.0,0.3,0.363636,0.5,0.000000,0.666667,1.0,0.50,0.5,1.0,1.0,0.000000,0.065586,1.0,0.000000,0.475234,0.251227,0.0,...,0.000000,0.0,0.497143,0.0,0.0,0.50,0.0,0.333333,0.5,0.5,0.166667,1.0,0.00,0.6,0.166667,0.694268,0.666667,0.4,0.340054,1.0,1.0,1.0,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.0,0.50,0.0,0.000000,0.000000,1.0,1.0,0.5,0.0,0.0,0.0,0.0


In [36]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [37]:
dataset_scaled.to_csv('X_test_HousePricePrediction_processed.csv')
!cp X_test_HousePricePrediction_processed.csv "drive/My Drive/"